This file created for data cleaning of participant 05.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from random import choice
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
p5data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Participant-05-detail.csv", sep=",")
p5data.dropna() #There should be a variable to storage data read from the csv file.

,Source,Date,Hour,Count
0,person5,2022-01-27,8,4
1,person5,2022-01-02,13,78
2,person5,2022-01-02,12,66
3,person5,2022-01-02,12,57
4,person5,2022-01-02,12,99
...,...,...,...,...
12132,person5,2022-09-01,19,59
12133,person5,2022-09-01,19,102
12134,person5,2022-09-01,19,15
12135,person5,2022-09-01,21,35


Now, I will use other data set to check wheather the model is good.(05)

In [5]:
p5data.sort_values(by='Date') #Ensure the data are in the correct order.
p5data = p5data.drop(['Source'], axis=1)
p5data

,Date,Hour,Count
0,2022-01-27,8,4
1,2022-01-02,13,78
2,2022-01-02,12,66
3,2022-01-02,12,57
4,2022-01-02,12,99
...,...,...,...
12132,2022-09-01,19,59
12133,2022-09-01,19,102
12134,2022-09-01,19,15
12135,2022-09-01,21,35


Group the time by weeks.

In [6]:
p5data.insert(0,'Week','')

In [7]:
p5data.groupby('Date')['Count'].sum()

Date
2022-01-01      114
2022-01-02     8681
2022-01-03     5104
2022-01-04     8031
2022-01-05     7743
              ...  
2022-08-28    12888
2022-08-29     6036
2022-08-30     9820
2022-08-31    10523
2022-09-01     6801
Name: Count, Length: 222, dtype: int64

In [8]:
p5data['Date'].replace('-','',regex=True,inplace=True)
p5data['Date'] = pd.to_datetime(p5data['Date'], format='%Y%m%d')

In [9]:
p5data['Week'] = p5data['Date'].dt.isocalendar().week # Calculate the wrrk number by python method.
p5data

,Week,Date,Hour,Count
0,4,2022-01-27,8,4
1,52,2022-01-02,13,78
2,52,2022-01-02,12,66
3,52,2022-01-02,12,57
4,52,2022-01-02,12,99
...,...,...,...,...
12132,35,2022-09-01,19,59
12133,35,2022-09-01,19,102
12134,35,2022-09-01,19,15
12135,35,2022-09-01,21,35


In [10]:
A = p5data.groupby('Week')['Count'].sum()

In [11]:
A.to_csv("/content/drive/MyDrive/Colab Notebooks/weeks_and_steps_05.csv",index=False,sep=',')

In [12]:
weeksteps05 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/weeks_and_steps_05.csv", sep=",")
weeksteps05.head() #There should be a variable to storage data read from the csv file.

,Count
0,32533
1,347
2,4
3,24943
4,48152


In [13]:
weeksteps05.insert(0,'Week','')

In [14]:
for n in range(0,len(weeksteps05)): 
  weeksteps05['Week'][n] = n+1 # Mark week numbers
weeksteps05

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Week,Count
0,1,32533
1,2,347
2,3,4
3,4,24943
4,5,48152
5,6,35990
6,7,49512
7,8,50478
8,9,64540
9,10,61896


In [15]:
weeksteps05.insert(2,'sedentary','')

In [16]:
weeksteps05['Count'] = weeksteps05['Count'].astype('int')

In [17]:
for a in range(0,len(weeksteps05)):
  if weeksteps05['Count'][a] >= 10000*7:
    weeksteps05['sedentary'][a] = 0
  else:
    weeksteps05['sedentary'][a] = 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
weeksteps05 = weeksteps05.drop([2])# Drop rows with incorrect values
weeksteps05.head()

,Week,Count,sedentary
0,1,32533,1
1,2,347,1
3,4,24943,1
4,5,48152,1
5,6,35990,1


In [19]:
weeksteps05.to_csv("/content/drive/MyDrive/Colab Notebooks/weeks_and_steps_5.csv",index=False,sep=',')

In [20]:
weeksteps05

,Week,Count,sedentary
0,1,32533,1
1,2,347,1
3,4,24943,1
4,5,48152,1
5,6,35990,1
6,7,49512,1
7,8,50478,1
8,9,64540,1
9,10,61896,1
10,11,90738,0
